In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'n2o': 6.4e-07}
band = [3]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.003685                    1  0.017184
0.000750        2  0.004072                    2  0.017491
0.001052        3  0.004873                    3  0.018141
0.001476        4  0.005690                    4  0.018794
0.002070        5  0.006513                    5  0.019456
0.002904        6  0.007337                    6  0.020117
0.004074        7  0.008163                    7  0.020791
0.005714        8  0.008989                    8  0.021467
0.008015        9  0.009770                    9  0.022145
0.011243       10  0.010529                   10  0.022830
0.015771       11  0.011225                   11  0.023510
0.022122       12  0.011866                   12  0.024198
0.031031       13  0.012431                   13  0.024885
0.043528       14  0.013127                   14  0.025776
0.061057       15  0.016649                   15  0.029002
0.085645       16  0.024445                   16  0.035735
0.120136       17  0.034203                   17  0.044211
0.168516       18  0.044614                   18  0.053399
0.236378       19  0.055646                   19  0.063287
0.331549       20  0.067287                   20  0.073877
0.465100       21  0.079574                   21  0.085198
0.652400       22  0.091155                   22  0.096030
0.915100       23  0.093577                   23  0.099605
1.283650       24  0.086701                   24  0.095549
1.800600       25  0.076848                   25  0.088480
2.525700       26  0.065235                   26  0.078977
3.542800       27  0.054558                   27  0.068928
4.969550       28  0.045589                   28  0.058961
6.970850       29  0.038117                   29  0.049006
9.778100       30  0.031718                   30  0.039129
13.715850      31  0.025871                   31  0.029357
19.239350      32  0.020119                   32  0.019916
26.987250      33  0.014142                   33  0.011012
37.855300      34  0.007636                   34  0.002685
53.100050      35 -0.001493                   35 -0.006597
73.887500      36 -0.012874                   36 -0.016520
97.662500      37 -0.021150                   37 -0.023098
121.437500     38 -0.021769                   38 -0.023211
145.212500     39 -0.017959                   39 -0.019602
168.987500     40 -0.014762                   40 -0.016250
192.762500     41 -0.011877                   41 -0.013107
216.537500     42 -0.009172                   42 -0.010129
240.312500     43 -0.006579                   43 -0.007267
264.087500     44 -0.004075                   44 -0.004508
287.862500     45 -0.001641                   45 -0.001834
311.637500     46  0.000734                   46  0.000770
335.412500     47  0.003057                   47  0.003301
359.187500     48  0.005336                   48  0.005781
382.962500     49  0.007572                   49  0.008208
406.737500     50  0.009770                   50  0.010588
430.512500     51  0.011934                   51  0.012924
454.287500     52  0.014067                   52  0.015221
478.062500     53  0.016171                   53  0.017492
501.837500     54  0.018251                   54  0.019728
525.612500     55  0.020308                   55  0.021932
549.387500     56  0.022347                   56  0.024108
573.162500     57  0.024370                   57  0.026269
596.937500     58  0.026382                   58  0.028408
620.712500     59  0.028376                   59  0.030525
644.487500     60  0.030343                   60  0.032603
668.262500     61  0.032244                   61  0.034602
692.037500     62  0.033971                   62  0.036439
715.812500     63  0.035388                   63  0.038001
739.587500     64  0.036459                   64  0.039269
763.362500     65  0.037327                   65  0.040357
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -36.248977  0.000000 -36.248977 -36.235337  4.384489e-08   
0.000624    2     -36.248980  0.000003 -36.248977 -36.235338  2.254771e-06   
0.000876    3     -36.248980  0.000004 -36.248977 -36.235338  3.150369e-06   
0.001229    4     -36.248982  0.000005 -36.248977 -36.235339  4.420409e-06   
0.001723    5     -36.248984  0.000007 -36.248976 -36.235339  6.221769e-06   
0.002417    6     -36.248986  0.000011 -36.248976 -36.235340  8.776363e-06   
0.003391    7     -36.248990  0.000015 -36.248975 -36.235342  1.239911e-05   
0.004757    8     -36.248995  0.000021 -36.248973 -36.235343  1.753720e-05   
0.006672    9     -36.249001  0.000030 -36.248971 -36.235346  2.482351e-05   
0.009359    10    -36.249011  0.000042 -36.248968 -36.235349  3.515547e-05   
0.013128    11    -36.249024  0.000060 -36.248964 -36.235354  4.980652e-05   
0.018415    12    -36.249042  0.000085 -36.248957 -36.235360  7.057732e-05   
0.025830    13    -36.249067  0.000120 -36.248946 -36.235368  1.000236e-04   
0.036232    14    -36.249101  0.000170 -36.248931 -36.235379  1.417643e-04   
0.050823    15    -36.249149  0.000241 -36.248908 -36.235394  2.011116e-04   
0.071291    16    -36.249212  0.000344 -36.248868 -36.235411  2.884068e-04   
0.100000    17    -36.249286  0.000501 -36.248785 -36.235423  4.226963e-04   
0.140271    18    -36.249367  0.000745 -36.248622 -36.235422  6.319403e-04   
0.196760    19    -36.249443  0.001120 -36.248323 -36.235390  9.571273e-04   
0.275997    20    -36.249493  0.001693 -36.247801 -36.235299  1.460902e-03   
0.387100    21    -36.249479  0.002563 -36.246915 -36.235104  2.238374e-03   
0.543100    22    -36.249324  0.003879 -36.245445 -36.234728  3.436110e-03   
0.761700    23    -36.248916  0.005832 -36.243084 -36.234060  5.255101e-03   
1.068500    24    -36.248219  0.008535 -36.239683 -36.233051  7.866556e-03   
1.498800    25    -36.247280  0.012016 -36.235264 -36.231719  1.140615e-02   
2.102400    26    -36.246116  0.016347 -36.229769 -36.230066  1.608004e-02   
2.949000    27    -36.244801  0.021573 -36.223227 -36.228153  2.208753e-02   
4.136600    28    -36.243398  0.027846 -36.215552 -36.226055  2.968761e-02   
5.802500    29    -36.241999  0.035443 -36.206556 -36.223902  3.917117e-02   
8.139200    30    -36.240792  0.044787 -36.196005 -36.221966  5.080078e-02   
11.417000   31    -36.240155  0.056465 -36.183690 -36.220749  6.477847e-02   
16.014700   32    -36.240799  0.071199 -36.169599 -36.221190  8.121049e-02   
22.464000   33    -36.244017  0.089788 -36.154229 -36.224933  1.001702e-01   
31.510500   34    -36.252086  0.113011 -36.139074 -36.234837  1.218764e-01   
44.200100   35    -36.269046  0.141449 -36.127597 -36.255787  1.468626e-01   
62.000000   36    -36.304293  0.173547 -36.130746 -36.297580  1.747443e-01   
85.775000   37    -36.372228  0.205225 -36.167002 -36.372868  2.035035e-01   
109.550000  38    -36.458767  0.232200 -36.226567 -36.463660  2.292380e-01   
133.325000  39    -36.553431  0.265556 -36.287875 -36.560387  2.605880e-01   
157.100000  40    -36.648373  0.309919 -36.338454 -36.656875  3.018651e-01   
180.875000  41    -36.742777  0.362748 -36.380030 -36.752506  3.517271e-01   
204.650000  42    -36.836119  0.422639 -36.413480 -36.846878  4.091823e-01   
228.425000  43    -36.928014  0.488702 -36.439312 -36.939693  4.734681e-01   
252.200000  44    -37.018088  0.560248 -36.457840 -37.030682  5.439873e-01   
275.975000  45    -37.106114  0.636798 -36.469317 -37.119616  6.202243e-01   
299.750000  46    -37.191923  0.717984 -36.473938 -37.206316  7.017590e-01   
323.525000  47    -37.275380  0.803508 -36.471872 -37.290637  7.882478e-01   
347.300000  48    -37.356377  0.893115 -36.463262 -37.372443  8.793517e-01   
371.075000  49    -37.434825  0.986590 -36.4482

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')